In [1]:
# Loading all the required packages
import pandas as pd
import numpy as np
# Packages for NLP
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [2]:
# Reading the dataset
df = pd.read_csv('fake_job_postings.csv', header = 0)
#.dropna('')
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [3]:
y=df['fraudulent'].value_counts()
y

0    17014
1      866
Name: fraudulent, dtype: int64

In [4]:
# Dropping unwanted features
df = df.drop(columns=['company_profile', 'description', 'requirements', 'benefits', 'telecommuting','has_company_logo','has_questions','department','salary_range'])
df.head()

,job_id,title,location,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [5]:
# Drop rows if 5 column value are missing
df = df.dropna(axis = 0, thresh=9)
df.shape

(7572, 9)

In [6]:
# Splitting Location Column
df_loc = df.location.str.split(",",expand = True)
df_loc.head()

,0,1,2,3,4,5,6
3,US,DC,Washington,None,None,None,None
4,US,FL,Fort Worth,None,None,None,None
6,DE,BE,Berlin,None,None,None,None
9,US,AZ,Phoenix,None,None,None,None
10,US,NJ,Jersey City,None,None,None,None


In [7]:
# Creating country column
df['country'] = df_loc[0]
df.head()

,job_id,title,location,employment_type,required_experience,required_education,industry,function,fraudulent,country
3,4,Account Executive - Washington DC,"US, DC, Washington",Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,US
4,5,Bill Review Manager,"US, FL, Fort Worth",Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,US
6,7,Head of Content (m/f),"DE, BE, Berlin",Full-time,Mid-Senior level,Master's Degree,Online Media,Management,0,DE
9,10,Customer Service Associate - Part Time,"US, AZ, Phoenix",Part-time,Entry level,High School or equivalent,Financial Services,Customer Service,0,US
10,11,ASP.net Developer Job opportunity at United St...,"US, NJ, Jersey City",Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology,0,US


In [8]:
import pycountry

In [9]:
# Renaming Null with Not Mentioned
#df['benefits'] = df['benefits'].replace(np.nan, 'Not Mentioned', regex = True)
df['employment_type'] = df['employment_type'].replace(np.nan, 'Not Mentioned', regex = True)
df['required_experience'] = df['required_experience'].replace(np.nan, 'Not Mentioned', regex = True)
df['industry'] = df['industry'].replace(np.nan, 'Not Mentioned', regex = True)
df['required_education'] = df['required_education'].replace(np.nan, 'Not Mentioned', regex = True)
df['function'] = df['function'].replace(np.nan, 'Not Mentioned', regex = True)
df['title'] = df['title'].replace(np.nan, 'Not Mentioned', regex = True)
df['location'] = df['location'].replace(np.nan, 'Not Mentioned', regex = True)
#df['company_profile'] = df['company_profile'].replace(np.nan, 'Not Mentioned', regex = True)
#df['description'] = df['description'].replace(np.nan, 'Not Mentioned', regex = True)
#df['requirements'] = df['requirements'].replace(np.nan, 'Not Mentioned', regex = True)
df['country'] = df['country'].replace(np.nan, 'Not Mentioned', regex = True)

In [10]:
# Mapping country name to country code
def findcountry (country_name):
    try:
        return pycountry.countries.get(alpha_2=country_name).name
    except: return (' ')
df['country_name'] = df['country'].apply(lambda row:findcountry(row))

df.head()

,job_id,title,location,employment_type,required_experience,required_education,industry,function,fraudulent,country,country_name
3,4,Account Executive - Washington DC,"US, DC, Washington",Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,US,United States
4,5,Bill Review Manager,"US, FL, Fort Worth",Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,US,United States
6,7,Head of Content (m/f),"DE, BE, Berlin",Full-time,Mid-Senior level,Master's Degree,Online Media,Management,0,DE,Germany
9,10,Customer Service Associate - Part Time,"US, AZ, Phoenix",Part-time,Entry level,High School or equivalent,Financial Services,Customer Service,0,US,United States
10,11,ASP.net Developer Job opportunity at United St...,"US, NJ, Jersey City",Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology,0,US,United States


In [11]:
# Drop country column
df = df.drop(columns=['country', 'location'])
#df.head(25)

In [12]:
# Merging all the textual column
descriptions = df['country_name'] + ' ' + df['employment_type'] +' '+df['required_education'] +' ' + df['function'].apply(str)
#Printing the first description
#print(descriptions)

In [13]:
# Create a function to vectorize the merged columns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import rbf_kernel
from nltk.tokenize import word_tokenize

def create_similarity_matrix(new_description, overall_descriptions):
#Append the new description to the overall set.
        # Define a tfidf vectorizer and remove all stopwords.
        tfidf = TfidfVectorizer(stop_words="english")
        #Convert tfidf matrix by fitting and transforming the data.
        tfidf_matrix = tfidf.fit_transform(overall_descriptions)
        # output the shape of the matrix.
        tfidf_matrix.shape
        # calculating the cosine similarity matrix.
        #cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)
        cosine_sim = cosine_similarity(tfidf_matrix,tfidf_matrix)
        
        return cosine_sim

In [14]:
# Create a function to sort the rows based on similarity values
def get_recommendations(new_description,overall_descriptions):
        # create the similarity matrix
        cosine_sim = create_similarity_matrix(new_description,overall_descriptions)
        # Get pairwise similarity scores of all the students with new student.
        sim_scores = list(enumerate(cosine_sim[-1]))
        # Sort the descriptions based on similarity score.
        sim_scores = sorted(sim_scores,key =lambda x:x[1],reverse= True )
        # Get the scores of top 10 descriptions.
        sim_scores = sim_scores[1:10]
#         # Get the student indices.
        indices = [i[0]for i in sim_scores]
        #return df_final.iloc[indices]
        #return df_final[['job_id' == indices]]
        return df_final.iloc[indices]
    #,sim_scores

In [ ]:
# country_name = 'United States'
# employment_type = "Full-time"
# #required_education = "Master's Degree"
# function = 'Engineering'
# title = 'Software'

# Taking the input of the user
country_name = input('Enter Country:' ) or 'Not Mentioned'
employment_type = input('Enter employment_type:') or 'Not Mentioned'
title = input('Enter Title:' ) or 'Not Mentioned'
function = input('Enter function:' ) or 'Not Mentioned'

# merging the input of the user
new_description = title + ' ' + function

# Taking the subset of dataset
df_final= df[df['country_name'] == country_name]
df_final= df_final[df_final['employment_type'] == employment_type]
#print(df_final)

# Passing the values to the above mentioned function
descriptions = df_final['title'] + ' ' + df_final['function']
print(new_description)
new_description = pd.Series(new_description)
get_recommendations(new_description,descriptions)